#### Modifying schema in custom fields

Custom field types can customise the schema generated for them using the `__modify_schema__` class method.

`__modify_schema__` can also take a `field` argument which will have type `Optional[ModelField]`. _pydantic_ will inspect the signature of `__modify_schema__` to determine whether the `field` argument should be included.

In [1]:
from typing import Any, Callable, Dict, Generator, Optional
from pydantic import BaseModel, Field
from pydantic.fields import ModelField

In [2]:
class RestrictedAlphabetStr(str):
    @classmethod
    def __get_validators__(cls) -> Generator[Callable, None, None]:
        yield cls.validate
    
    @classmethod
    def validate(cls, value: str, field: ModelField):
        alphabet = field.field_info.extra["alphabet"]
        if any(c not in alphabet for c in value):
            raise ValueError(f"{value!r} is not restricted to {alphabet!r}")
        return cls(value)
    
    @classmethod
    def __modify_schema__(
        cls, field_schema: Dict[str, Any], field: Optional[ModelField]
    ):
        if field:
            alphabet = field.field_info.extra["alphabet"]
            field_schema["examples"] = [c * 3 for c in alphabet]

In [3]:
class MyModel(BaseModel):
    value: RestrictedAlphabetStr = Field(alphabet="ABC")

In [4]:
print(MyModel.schema_json(indent=2))

{
  "title": "MyModel",
  "type": "object",
  "properties": {
    "value": {
      "title": "Value",
      "alphabet": "ABC",
      "examples": [
        "AAA",
        "BBB",
        "CCC"
      ],
      "type": "string"
    }
  },
  "required": [
    "value"
  ]
}
